In [1]:
import torch
from torch import nn
from torchvision import datasets, transforms

## 1. Parameters

In [2]:
# dataset
input_shape = 300
num_classes = 2

# hyper
batch_size = 4
num_epochs = 5
learning_rate = 0.001

# gpu
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 2. Dataset and Dataloader

In [3]:
pip install split_folders

Note: you may need to restart the kernel to use updated packages.


In [4]:
import splitfolders
raw = './raw'
split = './processed'
splitfolders.ratio(raw, split, seed = 1337, ratio = (0.7, 0.3))

ValueError: The provided input folder "./raw" does not exists. Your relative path cannot be found from the current working directory "C:\Users\12273\OneDrive\桌面\Git\AIHGP\OCT-Disease-Detection\codes\OCT-images-clasification".

In [ ]:
data_transfrom = transforms.Compose([  
    transforms.ToTensor(),             
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.Resize((300, 300)),
])
 
trainset = datasets.ImageFolder('./processed/train', transform = data_transfrom) 
testset = datasets.ImageFolder('./processed/val', transform = data_transfrom) 

classes = ('AMD', 'NORMAL')

In [ ]:
trainloader = torch.utils.data.DataLoader(dataset = trainset, 
                                          batch_size = batch_size, 
                                          shuffle = True, 
                                          num_workers = 1) 
testloader = torch.utils.data.DataLoader(dataset = testset, 
                                         batch_size = batch_size, 
                                         shuffle = True, 
                                         num_workers = 1)

In [ ]:
images, labels = next(iter(trainloader))

In [ ]:
images.shape

## 3. Model Arch

In [ ]:
class CNN(nn.Module):
    def __init__(self, input_shape, in_channels, num_classes):
        super(CNN, self).__init__()
        # conv2d: (b, 3, 300, 300) => (b, 9, 300, 300)
        # maxpool2d: (b, 9, 300, 300) => (b, 9, 150, 150)
        self.cnn1 = nn.Sequential(nn.Conv2d(in_channels = in_channels, out_channels = 9, 
                                            kernel_size = 5, padding = 2, stride = 1),
                                  nn.BatchNorm2d(9),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size = 2, stride = 2))
        # conv2d: (b, 9, 150, 150) => (b, 27, 150, 150)
        # maxpool2d: (b, 27, 150, 150) => (b, 27, 75, 75)
        self.cnn2 = nn.Sequential(nn.Conv2d(in_channels = 9, out_channels = 27, 
                                            kernel_size = 5, padding = 2, stride = 1),
                                  nn.BatchNorm2d(27),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size = 2, stride = 2))
        # (b, 27, 75, 75) => (b, 27*75*75)
        # (b, 27*75*75) => (b, 2)
        self.fc = nn.Linear(27*(input_shape//4)*(input_shape//4), num_classes)
        
    def forward(self, x):
        out = self.cnn1(x)
        out = self.cnn2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

### Torchsummary

In [ ]:
from torchsummary import summary

In [ ]:
model = CNN(input_shape = 300, in_channels = 3, num_classes = 2)

In [ ]:
summary(model, input_size = (3, 300, 300), batch_size = batch_size)

## 4. Model Train

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
total_batch = len(trainloader)

In [ ]:
for epoch in range(num_epochs):
    for batch_idx, (images, labels) in enumerate(trainloader):
#         images = images.to(device)
#         labels = labels.to(device)
        
        out = model(images)
        loss = criterion(out, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (batch_idx+1) % 10 == 0:
            print(f'{epoch+1}/{num_epochs}, {batch_idx+1}/{total_batch}: {loss.item():.4f}')

print('Finished Training')

## 5. Model Evaluation

In [ ]:
total = 0
correct = 0
for images, labels in testloader:
#    images = images.to(device)
#    labels = labels.to(device)
    out = model(images)
    preds = torch.argmax(out, dim = 1)
    
    total += images.size(0)
    correct += (preds == labels).sum().item()
print(f'{correct}/{total} = {correct/total}')

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for images, labels in testloader:
        out = model(images)
        _, predictions = torch.max(out, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

## 6. Model Save

In [ ]:
#torch.save(model.state_dict(),'cnn_1.ckpt')